In [ ]:
# battle_game_gui.py
"""
TurnQuest GUI — Turn-based battle system using Pygame.

Features:
- Player and enemy Characters with stats (HP, ATK, DEF, SPD)
- Moves with damage, healing, cooldowns, and status application (Burn)
- Items (Potion, Antidote) with quantities
- Simple enemy AI
- XP & leveling, small encounter loop
- GUI with buttons for moves and items, HUD, and a battle log

Install:
    pip install pygame

Run:
    python battle_game_gui.py
"""

import pygame
import random
import math
import sys
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Callable

# ----------------- Constants -----------------
WIDTH, HEIGHT = 960, 640
FPS = 30
FONT_NAME = None  # default pygame font
BG_COLOR = (20, 24, 30)
PANEL_COLOR = (30, 35, 45)
TEXT_COLOR = (230, 230, 230)
HP_BAR_BG = (60, 60, 60)
HP_BAR_FG = (110, 220, 120)
ENEMY_HP_COLOR = (220, 80, 80)
LOG_BG = (18, 20, 26)
BUTTON_COLOR = (50, 60, 75)
BUTTON_HOVER = (70, 90, 115)
DISABLED_COLOR = (60, 60, 60)
SMALL_PAD = 8
LARGE_PAD = 16

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("TurnQuest GUI")
clock = pygame.time.Clock()
font = pygame.font.Font(FONT_NAME, 18)
bigfont = pygame.font.Font(FONT_NAME, 28)

# ----------------- Data Classes -----------------
@dataclass
class Status:
    name: str
    duration: int
    potency: int = 0

@dataclass
class Move:
    name: str
    desc: str
    power: int = 0
    accuracy: float = 1.0
    cooldown: int = 0
    current_cd: int = 0
    target_self: bool = False
    apply_status: Optional[Callable[['Character','Character'], Optional[Status]]] = None

    def ready(self) -> bool:
        return self.current_cd <= 0

    def trigger_cd(self):
        self.current_cd = self.cooldown

    def tick_cd(self):
        if self.current_cd > 0:
            self.current_cd -= 1

@dataclass
class Item:
    name: str
    desc: str
    use: Callable[['Character'], str]
    qty: int = 1

@dataclass
class Character:
    name: str
    level: int
    max_hp: int
    hp: int
    atk: int
    defense: int
    speed: int
    moves: List[Move]
    statuses: List[Status] = field(default_factory=list)
    buffs: Dict[str, Status] = field(default_factory=dict)
    xp: int = 0

    def is_alive(self) -> bool:
        return self.hp > 0

    def take_damage(self, amount: int):
        amt = max(0, amount)
        self.hp = max(0, min(self.max_hp, self.hp - amt))
        return amt

    def heal(self, amount: int):
        healed = max(0, min(self.max_hp - self.hp, amount))
        self.hp += healed
        return healed

    def add_status(self, status: Status):
        for s in self.statuses:
            if s.name == status.name:
                s.duration = status.duration
                s.potency = status.potency
                return
        self.statuses.append(status)

    def get_stat(self, stat_name: str):
        base = getattr(self, stat_name)
        buff = self.buffs.get(stat_name)
        if buff:
            return max(0, base + buff.potency)
        return base

    def add_buff(self, stat: str, amount: int, duration: int):
        self.buffs[stat] = Status(name=f"buff:{stat}", duration=duration, potency=amount)

    def tick_statuses(self):
        msgs = []
        for s in self.statuses[:]:
            if s.name == 'Burn':
                dmg = s.potency
                self.take_damage(dmg)
                msgs.append(f"{self.name} takes {dmg} burn damage.")
            s.duration -= 1
            if s.duration <= 0:
                self.statuses.remove(s)
                msgs.append(f"{self.name} is no longer {s.name.lower()}.")
        for k, b in list(self.buffs.items()):
            b.duration -= 1
            if b.duration <= 0:
                del self.buffs[k]
                msgs.append(f"{self.name}'s {k} buff wore off.")
        return msgs

    def tick_move_cds(self):
        for m in self.moves:
            m.tick_cd()

# ----------------- Game Content -----------------
def make_player_moves():
    return [
        Move("Slash", "Solid strike.", power=18, accuracy=0.95, cooldown=0),
        Move("Power Strike", "Heavy, low accuracy.", power=30, accuracy=0.75, cooldown=2),
        Move("First Aid", "Heal yourself.", power=-30, accuracy=1.0, cooldown=3, target_self=True),
        Move("Flame Shot", "May burn target.", power=15, accuracy=0.9, cooldown=3,
             apply_status=lambda u, t: Status("Burn", duration=3, potency=5))
    ]

def make_enemy_moves(level):
    return [
        Move("Bite", "A bite.", power=12 + level * 2, accuracy=0.95),
        Move("Rage", "Powerful blow.", power=20 + level * 3, accuracy=0.8, cooldown=2),
        Move("Spit", "Weak hit, may burn.", power=8 + level, accuracy=0.85,
             apply_status=lambda u, t: Status("Burn", duration=3, potency=4))
    ]

def make_items():
    return {
        'potion': Item("Potion", "Heals 40 HP.", qty=3, use=lambda c: f"{c.name} healed {c.heal(40)} HP."),
        'antidote': Item("Antidote", "Removes Burn.", qty=1, use=lambda c: antidote_use(c))
    }

def antidote_use(c: Character):
    removed = False
    for s in c.statuses[:]:
        if s.name == 'Burn':
            c.statuses.remove(s)
            removed = True
    return f"Removed burn." if removed else "No effect."

def calculate_damage(attacker: Character, defender: Character, base_power: int):
    atk = attacker.get_stat('atk')
    defense = defender.get_stat('defense')
    if defense <= 0:
        defense = 1
    raw = (atk * base_power) / defense
    variation = random.uniform(0.85, 1.15)
    dmg = int(max(1, raw * variation))
    return dmg

def attempt_hit(move: Move):
    return random.random() <= move.accuracy

def enemy_choose_move(enemy: Character, player: Character):
    usable = [m for m in enemy.moves if m.ready()]
    if not usable:
        return None
    if enemy.hp < enemy.max_hp * 0.35:
        candidates = sorted(usable, key=lambda mm: mm.power)
        return candidates[0]
    candidates = sorted(usable, key=lambda mm: mm.power, reverse=True)
    if random.random() < 0.2:
        for m in usable:
            if m.apply_status:
                return m
    return candidates[0]

def generate_enemy(player_level: int) -> Character:
    lvl = max(1, player_level + random.choice([-1, 0, 1]))
    base_hp = 50 + lvl * 8
    enemy = Character(
        name=random.choice(['Goblin', 'Wolf', 'Bandit', 'Slime', 'Rogue']),
        level=lvl,
        max_hp=base_hp,
        hp=base_hp,
        atk=8 + lvl * 2,
        defense=4 + lvl,
        speed=5 + lvl,
        moves=make_enemy_moves(lvl)
    )
    return enemy

# ----------------- GUI Helpers -----------------
def draw_text(surf, text, x, y, fontobj, color=TEXT_COLOR, align="topleft"):
    txt = fontobj.render(text, True, color)
    rect = txt.get_rect()
    setattr(rect, align, (x, y))
    surf.blit(txt, rect)
    return rect

class Button:
    def __init__(self, rect, text, callback, fontobj=font, tooltip=None):
        self.rect = pygame.Rect(rect)
        self.text = text
        self.callback = callback
        self.font = fontobj
        self.tooltip = tooltip
        self.enabled = True

    def draw(self, surf, mouse_pos):
        col = BUTTON_COLOR if self.enabled else DISABLED_COLOR
        if self.rect.collidepoint(mouse_pos) and self.enabled:
            col = BUTTON_HOVER
        pygame.draw.rect(surf, col, self.rect, border_radius=6)
        draw_text(surf, self.text, self.rect.centerx, self.rect.centery, self.font, color=TEXT_COLOR, align="center")

    def handle_event(self, event, mouse_pos):
        if not self.enabled:
            return
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1 and self.rect.collidepoint(mouse_pos):
            self.callback()

# ----------------- Game GUI State -----------------
class Game:
    def __init__(self):
        self.player = None
        self.enemy = None
        self.inventory = None
        self.log: List[str] = []
        self.buttons: List[Button] = []
        self.item_buttons: List[Button] = []
        self.move_buttons: List[Button] = []
        self.in_battle = False
        self.player_turn = True
        self.message_cooldown = 0.0
        self.encounters = 0

        # Start new player
        self.start_new_player()

    def start_new_player(self):
        name = self.prompt_player_name()
        self.player = Character(
            name=name or "Hero",
            level=1, max_hp=100, hp=100,
            atk=12, defense=8, speed=10,
            moves=make_player_moves()
        )
        self.inventory = make_items()
        self.start_encounter()

    def prompt_player_name(self):
        # Simple blocking prompt using console to ask name (keeps GUI code short)
        pygame.display.iconify()
        name = input("Choose your hero's name (leave blank for 'Hero'): ").strip()
        pygame.display.set_mode((WIDTH, HEIGHT))
        return name

    def start_encounter(self):
        self.enemy = generate_enemy(self.player.level)
        self.encounters += 1
        self.log = [f"A wild {self.enemy.name} (Lv {self.enemy.level}) appears!"]
        self.in_battle = True
        self.player_turn = True
        # reset move cds? keep current
        self.create_buttons()

    def create_buttons(self):
        # Create move buttons (left side)
        self.buttons = []
        self.move_buttons = []
        self.item_buttons = []
        left_x = 20
        move_w = 220
        move_h = 44
        y = HEIGHT - (move_h + LARGE_PAD) * 1 - 120
        for idx, move in enumerate(self.player.moves):
            rect = (left_x, y + idx * (move_h + SMALL_PAD), move_w, move_h)
            btn = Button(rect, f"{move.name} {'(CD:'+str(move.current_cd)+')' if move.current_cd>0 else ''}", 
                         callback=lambda m=move: self.player_use_move(m))
            self.move_buttons.append(btn)
            self.buttons.append(btn)
        # Items (right of moves)
        x2 = left_x + move_w + 20
        for i, key in enumerate(self.inventory.keys()):
            it = self.inventory[key]
            rect = (x2, y + i * (move_h + SMALL_PAD), 180, move_h)
            btn = Button(rect, f"{it.name} x{it.qty}", callback=lambda k=key: self.player_use_item(k))
            self.item_buttons.append(btn)
            self.buttons.append(btn)
        # Pass button
        pass_rect = (x2 + 200, y + 0, 120, move_h)
        self.buttons.append(Button(pass_rect, "Pass", callback=self.player_pass))
        # Rest button (small heal between fights)
        # We'll handle rest on end-of-battle screen; not as a button here.

    def refresh_buttons_text(self):
        for btn, move in zip(self.move_buttons, self.player.moves):
            cd = f"(CD:{move.current_cd})" if move.current_cd > 0 else ""
            btn.text = f"{move.name} {cd}"

        for i, key in enumerate(self.inventory.keys()):
            it = self.inventory[key]
            if i < len(self.item_buttons):
                self.item_buttons[i].text = f"{it.name} x{it.qty}"
                self.item_buttons[i].enabled = it.qty > 0

        for b in self.move_buttons:
            # enable only on player's turn and if move ready and player alive
            idx = self.move_buttons.index(b)
            m = self.player.moves[idx]
            b.enabled = self.player_turn and self.in_battle and self.player.is_alive() and m.ready()

    def push_log(self, text: str):
        self.log.insert(0, text)
        if len(self.log) > 8:
            self.log = self.log[:8]

    def player_use_move(self, move: Move):
        if not self.player_turn or not self.in_battle:
            return
        if not move.ready():
            self.push_log(f"{move.name} is on cooldown.")
            return
        # resolve hit
        self.push_log(f"{self.player.name} uses {move.name}!")
        if not attempt_hit(move):
            self.push_log("It missed!")
            move.trigger_cd()
            self.end_player_action()
            return
        if move.power < 0:
            amt = -move.power
            healed = self.player.heal(amt)
            self.push_log(f"{self.player.name} healed {healed} HP.")
        else:
            dmg = calculate_damage(self.player, self.enemy, move.power)
            inflicted = self.enemy.take_damage(dmg)
            self.push_log(f"{move.name} dealt {inflicted} damage to {self.enemy.name}.")
        if move.apply_status:
            s = move.apply_status(self.player, self.enemy)
            if s:
                self.enemy.add_status(s)
                self.push_log(f"{self.enemy.name} afflicted with {s.name}.")
        move.trigger_cd()
        self.end_player_action()

    def player_use_item(self, key: str):
        if not self.player_turn or not self.in_battle:
            return
        it = self.inventory[key]
        if it.qty <= 0:
            self.push_log("No item left.")
            return
        self.push_log(it.use(self.player))
        it.qty -= 1
        self.end_player_action()

    def player_pass(self):
        if not self.player_turn or not self.in_battle:
            return
        self.push_log(f"{self.player.name} passes.")
        self.end_player_action()

    def end_player_action(self):
        # tick statuses at end of player's action? We'll do start-of-turn ticks.
        self.player.tick_move_cds()
        self.player_turn = False
        # small delay handled by message cooldown to give time to see message
        self.message_cooldown = 0.35

    def enemy_take_turn(self):
        if not self.enemy.is_alive() or not self.player.is_alive():
            return
        move = enemy_choose_move(self.enemy, self.player)
        if not move:
            self.push_log(f"{self.enemy.name} cannot act.")
            return
        self.push_log(f"{self.enemy.name} uses {move.name}!")
        if not attempt_hit(move):
            self.push_log("It missed!")
            move.trigger_cd()
            return
        if move.power < 0:
            amt = -move.power
            healed = self.enemy.heal(amt)
            self.push_log(f"{self.enemy.name} healed {healed} HP.")
        else:
            dmg = calculate_damage(self.enemy, self.player, move.power)
            inflicted = self.player.take_damage(dmg)
            self.push_log(f"{self.enemy.name} dealt {inflicted} damage to {self.player.name}.")
        if move.apply_status:
            s = move.apply_status(self.enemy, self.player)
            if s:
                self.player.add_status(s)
                self.push_log(f"{self.player.name} afflicted with {s.name}.")
        move.trigger_cd()

    def tick_turn_start(self):
        # called at start of round: statuses tick
        msgs = self.player.tick_statuses()
        for m in msgs:
            self.push_log(m)
        msgs = self.enemy.tick_statuses()
        for m in msgs:
            self.push_log(m)
        # disable move buttons until player's turn is true
        self.player_turn = True
        self.player.tick_move_cds()
        self.enemy.tick_move_cds()
        self.refresh_buttons_text()

    def update(self, dt):
        # Called every frame, dt in seconds
        if not self.in_battle:
            return

        if not self.player_turn:
            # Allow a small pause so player sees action results
            if self.message_cooldown > 0:
                self.message_cooldown -= dt
                return
            # Enemy acts
            self.enemy_take_turn()
            # Check death
            if not self.player.is_alive() or not self.enemy.is_alive():
                self.handle_battle_end()
                return
            # Next round starts
            self.tick_turn_start()
            return

        # check win/loss every frame
        if not self.enemy.is_alive() or not self.player.is_alive():
            self.handle_battle_end()

        self.refresh_buttons_text()

    def handle_battle_end(self):
        if self.player.is_alive() and not self.enemy.is_alive():
            self.push_log(f"Victory! {self.enemy.name} defeated.")
            xp_gain = self.enemy.level * 25
            self.player.xp += xp_gain
            self.push_log(f"You gain {xp_gain} XP.")
            self.check_level_up()
            # item drops
            if random.random() < 0.4:
                self.inventory['potion'].qty += 1
                self.push_log("You found a Potion!")
            if random.random() < 0.15:
                self.inventory['antidote'].qty += 1
                self.push_log("You found an Antidote!")
            self.in_battle = False
            # simple rest prompt handled by console or automatic
            self.post_battle_prompt()
        elif not self.player.is_alive():
            self.push_log("You were defeated...")
            self.in_battle = False
            self.push_log("Game over. Restarting a new hero...")
            pygame.time.set_timer(pygame.USEREVENT + 1, 1500, loops=1)  # schedule restart

    def check_level_up(self):
        leveled = False
        while self.player.xp >= 100:
            self.player.xp -= 100
            self.player.level += 1
            leveled = True
            hp_gain = random.randint(6, 12)
            atk_gain = random.randint(1, 3)
            def_gain = random.randint(1, 3)
            spd_gain = random.randint(0, 2)
            self.player.max_hp += hp_gain
            self.player.atk += atk_gain
            self.player.defense += def_gain
            self.player.speed += spd_gain
            self.player.hp = self.player.max_hp
            self.push_log(f"*** {self.player.name} leveled up to Lv {self.player.level}! ***")
            self.push_log(f"+{hp_gain} HP, +{atk_gain} ATK, +{def_gain} DEF, +{spd_gain} SPD")
        return leveled

    def post_battle_prompt(self):
        # Blocking simple choices via console to keep GUI code small:
        pygame.display.iconify()
        ans = input("Encounter finished. (R)est +30 HP, (C)ontinue, or (Q)uit? ").strip().lower()
        pygame.display.set_mode((WIDTH, HEIGHT))
        if ans.startswith('r'):
            healed = self.player.heal(30)
            self.push_log(f"You rest and recover {healed} HP.")
            # continue to next encounter
            self.start_encounter()
        elif ans.startswith('q'):
            print("Quitting game. Thanks for playing.")
            pygame.quit()
            sys.exit(0)
        else:
            # continue
            self.start_encounter()

    def draw(self, surf):
        surf.fill(BG_COLOR)
        # Top panel: Player and Enemy HUD
        panel_h = 140
        pygame.draw.rect(surf, PANEL_COLOR, (12, 12, WIDTH - 24, panel_h), border_radius=8)
        # Player HUD (left)
        ph_x, ph_y = 32, 32
        draw_text(surf, f"{self.player.name} (Lv {self.player.level})", ph_x, ph_y, bigfont)
        draw_text(surf, f"HP: {self.player.hp}/{self.player.max_hp}", ph_x, ph_y + 38, font)
        self.draw_hp_bar(surf, ph_x, ph_y + 68, 340, self.player.hp, self.player.max_hp, HP_BAR_FG)
        draw_text(surf, f"ATK: {self.player.get_stat('atk')}  DEF: {self.player.get_stat('defense')}  SPD: {self.player.get_stat('speed')}", ph_x, ph_y + 92, font)
        draw_text(surf, "Statuses: " + (", ".join([f"{s.name}({s.duration})" for s in self.player.statuses]) or "None"), ph_x + 400, ph_y + 12, font)

        # Enemy HUD (right)
        eh_x = WIDTH - 420
        draw_text(surf, f"{self.enemy.name} (Lv {self.enemy.level})", eh_x, ph_y, bigfont)
        draw_text(surf, f"HP: {self.enemy.hp}/{self.enemy.max_hp}", eh_x, ph_y + 38, font)
        self.draw_hp_bar(surf, eh_x, ph_y + 68, 340, self.enemy.hp, self.enemy.max_hp, ENEMY_HP_COLOR)
        draw_text(surf, "Statuses: " + (", ".join([f"{s.name}({s.duration})" for s in self.enemy.statuses]) or "None"), eh_x + 0, ph_y + 92, font)

        # Center: enemy "sprite" (simple rectangle)
        sprite_rect = pygame.Rect(WIDTH//2 - 80, panel_h + 40, 160, 160)
        pygame.draw.rect(surf, (120, 80, 60), sprite_rect, border_radius=12)
        draw_text(surf, self.enemy.name, sprite_rect.centerx, sprite_rect.y - 8, font, align="center")

        # Bottom area: buttons & log
        # Draw move/item buttons
        mouse_pos = pygame.mouse.get_pos()
        for b in self.buttons:
            b.draw(surf, mouse_pos)

        # Draw log
        log_x = WIDTH - 340
        log_y = panel_h + 20
        log_w = 300
        log_h = 260
        pygame.draw.rect(surf, LOG_BG, (log_x, log_y, log_w, log_h), border_radius=8)
        # recent messages top-down
        for i, line in enumerate(self.log[:8]):
            draw_text(surf, line, log_x + 10, log_y + 12 + i * 28, font)

        # Bottom left instructions
        draw_text(surf, "Click moves or items. When your turn ends, enemy acts.", 28, HEIGHT - 36, font)

        # XP bar and info
        xp_txt = f"XP: {self.player.xp}/100"
        draw_text(surf, xp_txt, WIDTH//2 - 40, 26, font)

    def draw_hp_bar(self, surf, x, y, width, cur, mx, color):
        pygame.draw.rect(surf, HP_BAR_BG, (x, y, width, 16), border_radius=8)
        ratio = max(0.0, min(1.0, cur / mx if mx > 0 else 0))
        pygame.draw.rect(surf, color, (x, y, int(width * ratio), 16), border_radius=8)

    def handle_event(self, event):
        mouse_pos = pygame.mouse.get_pos()
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            for b in self.buttons:
                b.handle_event(event, mouse_pos)
        if event.type == pygame.USEREVENT + 1:
            # restart after defeat
            self.start_new_player()

# ----------------- Main loop -----------------
def main():
    game = Game()
    # initial tick to set buttons
    game.create_buttons()
    game.tick_turn_start()

    running = True
    while running:
        dt = clock.tick(FPS) / 1000.0
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            game.handle_event(event)

        game.update(dt)
        game.draw(screen)
        pygame.display.flip()

    pygame.quit()
    sys.exit(0)

if __name__ == "__main__":
    main()
